In [3]:
import torch
import copy
import numpy as np
from torch import nn

import sys
sys.path.append('.')
sys.path.append('..')

from algorithms.iterative_relaxation import IterativeRelaxation
from algorithms.decision_procedure import MarabouCoreDP

from models.test_models import ProphecyPaperNetwork, TestModel
from models.utils import attach_relu_activation_hook, attach_layer_output_hook, get_layers_info
from models.utils import turn_bool_activation_to_str

### Test Improved DP

In [4]:
improved_dp = MarabouCoreDP()
prophecy_model = ProphecyPaperNetwork()

In [5]:
get_layers_info(TestModel())

[{'name': 'model_input', 'in_features': 2, 'out_features': 2, 'layer': None},
 {'name': 'sequential_stack.0',
  'in_features': 2,
  'out_features': 5,
  'layer': Linear(in_features=2, out_features=5, bias=False)},
 {'name': 'sequential_stack.1',
  'in_features': 5,
  'out_features': 5,
  'layer': ReLU()},
 {'name': 'sequential_stack.2',
  'in_features': 5,
  'out_features': 3,
  'layer': Linear(in_features=5, out_features=3, bias=False)},
 {'name': 'sequential_stack.3',
  'in_features': 3,
  'out_features': 3,
  'layer': ReLU()},
 {'name': 'final_output',
  'in_features': 3,
  'out_features': 2,
  'layer': Linear(in_features=3, out_features=2, bias=False)}]

In [8]:
activation = {
  'linear_relu_stack.1': ['ON', 'OFF'], 
  'linear_relu_stack.3': ['ON', 'OFF'],
}
input_ranges = [[-100, 100], [-100, 100]]
specification = [(np.array([[1, -1]]), np.array([0]))] # class = y1
# specification = [(np.array([[-1, 1]]), np.array([0]))] # class = y2
improved_dp.solve(activation, prophecy_model, input_ranges, specification)

Initiating input query...


['unsat',
 {},

In [9]:
activation = {
  'linear_relu_stack.1': ['ON', 'OFF'], 
  'linear_relu_stack.3': ['--', '--'],
}
input_ranges = [[-100, 100], [-100, 100]]
specification = [(np.array([[1, -1]]), np.array([0]))] # class = y1
# specification = [(np.array([[-1, 1]]), np.array([0]))] # class = y2
improved_dp.solve(activation, prophecy_model, input_ranges, specification)

Initiating input query...


['unsat',
 {},

In [10]:
activation = {
  'linear_relu_stack.1': ['--', '--'], 
  'linear_relu_stack.3': ['ON', '--'],
}
input_ranges = [[-100, 100], [-100, 100]]
specification = [(np.array([[1, -1]]), np.array([0]))]
# specification = [(np.array([[-1, 1]]), np.array([0]))] # class = y2
improved_dp.solve(activation, prophecy_model, input_ranges, specification)

Initiating input query...


['sat',
 {0: -49.999999,
  1: -50.000001,
  2: 1.9999999949504854e-06,
  3: -100.0,
  4: 1.9999999949504854e-06,
  5: 0.0,
  6: 9.999999974752427e-07,
  7: -9.999999974752427e-07,
  8: 9.999999974752427e-07,
  9: 9.999999974752427e-07,
  10: -0.0,
  11: -0.0},

### Test Iterative Relaxation

In [11]:
ir_model = ProphecyPaperNetwork()
iterative_relaxation = IterativeRelaxation()

input_data = [[1, -1]]
input_ranges = [[-100, 100], [-100, 100]]
specification = [(np.array([[1, -1]]), np.array([0]))] # class = y1
# specification = [(np.array([[-1, 1]]), np.array([0]))] # class = y2

input_property = iterative_relaxation.call(ir_model, input_data, input_ranges, specification)
print(f"input property: {input_property}")

Initiating input query...
unconstrained_layer: linear_relu_stack.3
{'linear_relu_stack.1': ['ON', 'OFF'], 'linear_relu_stack.3': ['--', '--']}
Initiating input query...
unconstrained_layer: linear_relu_stack.1
{'linear_relu_stack.1': ['--', '--'], 'linear_relu_stack.3': ['--', '--']}
Initiating input query...
Critical layer found: 0
--- unconstraining neuron 0 in critical layer
Initiating input query...
--- neuron needed
--- unconstraining neuron 1 in critical layer
Initiating input query...
--- neuron needed
input property: [{'linear_relu_stack.1': ['ON', 'OFF'], 'linear_relu_stack.3': ['--', '--']}]


In [12]:
ir_model = ProphecyPaperNetwork()
iterative_relaxation = IterativeRelaxation()

input_data = [[1, -1]]
input_ranges = [[-100, 100], [-100, 100]]
specification = [(np.array([[-1, 1]]), np.array([0]))] # class = y2

input_property = iterative_relaxation.call(ir_model, input_data, input_ranges, specification)
print(f"input property: {input_property}")

Initiating input query...
input property: [{'linear_relu_stack.1': ['ON', 'OFF'], 'linear_relu_stack.3': ['ON', 'OFF']}, [(array([[-1,  1]]), array([0]))]]


### Misc

In [ ]:
# model = ProphecyPaperNetwork()
# _act_handles, activation_signature = attach_relu_activation_hook(model)
# _out_handles, layer_outputs = attach_layer_output_hook(model)

# # input_data = [[-1,3]]
# input_data = [[1, -1]]
# # input_data = [[-10, -0]]

# X = torch.tensor(input_data, dtype=torch.float)
# logits = model(X)
# prediction_prob = nn.Softmax(dim=1)(logits)
# y_pred = prediction_prob.argmax(1)

# activation_signature = turn_bool_activation_to_str(activation_signature)

# print(f"Predicted class: {y_pred}")
# print(f"Activation: {activation_signature}")
# print(f"Layer outputs: {layer_outputs}")